In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Đọc dữ liệu từ file CSV
urls = pd.read_csv('brands_data.csv')
brands_check = pd.read_csv('phone_brands.csv')
brands_checkok = []
for i in brands_check['Brand'].tolist():
    brands_checkok.append(i.lower())


count = 0
names = []
linkss = []
brandsss = []  # Danh sách để lưu tên thương hiệu

for url,name in zip(urls['Link'], urls['Brand']):
    if name.lower() not in brands_checkok:
        continue
    # if count == 2:
    #     break

    parts = url.split('/')
    parts = parts[-1].split('-')
    brand_name = parts[0].capitalize()  # Lấy tên thương hiệu từ URL
    id = parts[-1].split('.')[0]
    url_brands = [url]
    print(brand_name)

    try:
        # Gửi yêu cầu GET để lấy HTML của trang
        response = requests.get(url)
        response.raise_for_status()  # Kiểm tra nếu yêu cầu thất bại

        # Phân tích cú pháp HTML của trang
        soup = BeautifulSoup(response.content, 'html.parser')
        # Tìm phần tử chứa liên kết đến các trang pate
        nav_pages = soup.find('div', {'class': 'nav-pages'})

        # Nếu tìm thấy phần tử nav_pages
        if nav_pages:
            max = 0
            for link in nav_pages.find_all('a'):
                href = link.get('href')
                if href and 'p' in href:  # Kiểm tra nếu là liên kết trang
                    full_link = f'https://m.gsmarena.com/{href}'
                    match = re.search(r'p(\d+)', full_link)
                    page_number = match.group(1)
                    if int(page_number) > max:
                        max = int(page_number)
                    if full_link not in url_brands:
                        url_brands.append(full_link)
        # Tạo các link cho pate từ 2 đến 10
        for i in range(2, max + 1):
            url_p = f'https://m.gsmarena.com/{parts[0]}-phones-f-{id}-0-p{i}.php'
            if url_p not in url_brands:
                url_brands.append(url_p)
        
    except Exception as e:
        print("Lỗi lấy page:", e)

    for url_brand in url_brands:
        print(url_brand)
        try:
            # Gửi yêu cầu GET để lấy HTML của trang
            response = requests.get(url_brand)
            response.raise_for_status()  # Kiểm tra nếu yêu cầu thất bại

            # Phân tích cú pháp HTML của trang
            soup = BeautifulSoup(response.content, 'html.parser')
            # Tìm phần tử chứa danh sách các brand
            brands_list = soup.find('div', {'class': 'makers'})

            # Tìm phần tử chứa danh sách các điện thoại
            phones_list = brands_list.find_all('li')  # Lấy tất cả thẻ <li>

            # Duyệt qua các thẻ <li> để lấy tên và liên kết của từng điện thoại
            for phone in phones_list:
                link_tag = phone.find('a')
                if link_tag:
                    phone_name = link_tag.find('strong').text.strip()
                    phone_href = link_tag['href']

                    # Lưu vào danh sách
                    names.append(phone_name)
                    linkss.append(f'https://www.gsmarena.com/{phone_href}')
                    brandsss.append(brand_name)  # Lưu tên thương hiệu
        except Exception as e:
            print("Error1:", e)

        try:
            # Gửi yêu cầu GET để lấy HTML của trang
            response = requests.get(url_brand)
            response.raise_for_status()  # Kiểm tra nếu yêu cầu thất bại

            # Phân tích cú pháp HTML của trang
            soup = BeautifulSoup(response.content, 'html.parser')
            # Tìm phần tử chứa danh sách các brand
            brands_list = soup.find('div', {'class': 'general-menu'})

            # Tìm phần tử chứa danh sách các điện thoại
            phones_list = brands_list.find_all('li')  # Lấy tất cả thẻ <li>

            # Duyệt qua các thẻ <li> để lấy tên và liên kết của từng điện thoại
            for phone in phones_list:
                link_tag = phone.find('a')
                if link_tag:
                    phone_name = link_tag.find('strong').text.strip()
                    phone_href = link_tag['href']

                    # Lưu vào danh sách
                    names.append(phone_name)
                    linkss.append(f'https://www.gsmarena.com/{phone_href}')
                    brandsss.append(brand_name)  # Lưu tên thương hiệu
        except Exception as e:
            print("Error2:", e)

    count += 1

# Tạo DataFrame từ danh sách
data = pd.DataFrame({
    'Brand': brandsss,
    'Phone Name': names,
    'Link': linkss
})

# Lưu DataFrame vào file CSV
data.to_csv('phones_data1.csv', index=False, encoding='utf-8-sig')
print("Dữ liệu đã được lưu vào phones_data.csv")


Apple
https://www.gsmarena.com/apple-phones-48.php
Error2: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/apple-phones-f-48-0-p2.php
Error1: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/apple-phones-f-48-0-p3.php
Error1: 'NoneType' object has no attribute 'find_all'
Asus
https://www.gsmarena.com/asus-phones-46.php
Error2: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/asus-phones-f-46-0-p2.php
Error1: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/asus-phones-f-46-0-p3.php
Error1: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/asus-phones-f-46-0-p4.php
Error1: 'NoneType' object has no attribute 'find_all'
Google
https://www.gsmarena.com/google-phones-107.php
Error2: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/google-phones-f-107-0-p2.php
Error1: 'NoneType' object has no attribute 'find_all'
https://m.gsmarena.com/google-phones-f-107-0-p3.php
Error1: '

In [38]:
import pandas as pd

# Load CSV file (you need to specify the correct path to your file)
file_path = "phones_data1.csv"
df = pd.read_csv(file_path)

# Combine 'brand' and 'product_name' columns and convert to lowercase
df['combined'] = (df['Brand'] + ' ' + df['Phone Name']).str.lower()
# Save the result to a new CSV file
output_file = "phones_dict_check.csv"
df.to_csv(output_file, index=False)

print(f"File saved as {output_file}")


File saved as phones_dict_check.csv
